In [1]:
### This is about simulating a great number of times the Velib trafic in Rouen
import Distributions
using Distributions
srand(123);

In [2]:
#General Data
T = 1000 #horizon
n = 5 #number of stations : 3-7
dist_matrix = [0 3 5 7 7;
               2 0 2 5 5;
               4 2 0 3 3;
               8 6 4 0 2;
               7 7 5 2 0]; #distance between stations
terminal_number = [24 20 20 15 20]; #number of terminals at each station
routing_matrix = [0 0.2 0.3 0.2 0.3;
                  0.2 0 0.3 0.2 0.3;
                  0.2 0.25 0 0.25 0.3;
                  0.15 0.2 0.3 0 0.35;
                  0.2 0.25 0.35 0.2 0]; #routing matrix ?
leaving_per_hour = [2.8 3.7 5.5 3.5 4.6]; #mean number of bikes leaving each station per hour
alpha = 0.1;
N_bikes = convert(Integer, 91); #total number of bikes
N_stations = convert(Integer, 5); #number of stations

In [3]:
#Initial data
X0 = [20 1 0 0 0;
     1 15 1 0 0;
     0 1 17 1 0;
     0 0 1 13 1;
     0 0 0 1 18];

In [4]:
function get_event(v, P)
    i0 = 1; j0 = 1; s = 0;
    for i in range(1, N_stations)
        for j in range(1, N_stations)
            s = s + P[(i-1)*N_stations + j];
            if P[(i-1)*N_stations + j] != 0
                if v > s
                    i0 = i; j0 = j;
                else
                    return [i0, j0];
                end
            end
        end
    end
    return [i0, j0]
end

get_event (generic function with 1 method)

In [5]:
function simulate(T)
    #define distributions
    c = Uniform();
    route = [Categorical(routing_matrix[i, :]) for i in range(1, N_stations)]; #TO BE CHANGED
    #time
    t = 0;
    #state
    X = X0;
    #number of problems
    result = 0;
    while (t < T)
        #********************************************************************
        #Get the parameter of the stay time Exponential 
        #get also the vector with probabilities of each event p
        lambda = 0;
        p = zeros(N_stations*N_stations);
        for i in range(1, N_stations)
            for j in range(1, N_stations)
                if i != j
                    lambda = lambda + X[i,j]*dist_matrix[i,j];
                    p[N_stations*(i-1) + j] = X[i,j]*dist_matrix[i,j];
                else
                    lambda = lambda + alpha * leaving_per_hour[i];
                    p[N_stations*(i-1) + j] = alpha * leaving_per_hour[i];
                end
            end
        end
        p = p./lambda;
        
        #*****************************************************************
        #Draw the time to stay
        exp = Exponential(lambda);
        dt = rand(exp);
        t = t + dt;
        
        #*****************************************************************
        #draw event and realise it
        cat = rand(c);
        A = get_event(cat, p); i = A[1]; j = A[2];
        #if i == j it is a departure. Else it is an arrival.
        if i == j
            #if there is no bike available
            if X[i,j] == 0
                result = result + 1;
            else
                X[i,j] = X[i,j] - 1;
                destination = rand(route[i]);
                X[i, destination] = X[i, destination] + 1;
            end
            println("departure ", t);
        else #it is an arrival
            #if no terminal is available
            if X[j,j] == terminal_number[j]
                result = result + 1;
            else
                X[i,j] = X[i,j] - 1;
                X[j,j] = X[j,j] + 1;
            end
            println("arrival ", t);
        end
    end
    return result
end

simulate (generic function with 1 method)

In [6]:
simulate(1000)

arrival 2.79289154812082
arrival 12.97664403880918
arrival 26.484082606811214
departure 32.53640049749328
departure 34.76868665046818
arrival 66.49058364284906
arrival 78.3164982289261
departure 86.27764884065121
departure 119.0173793571536
departure 124.53503186802475
departure 142.82153670821612
departure 210.5748040337932
arrival 220.83509716361283
arrival 323.50374903815265
departure 325.47881687075136
arrival 332.36684012821826
departure 357.79593614950903
departure 396.56242873752484
departure 400.91937636089506
departure 468.2876748748473
departure 529.0004676247481
arrival 543.7177373891963
arrival 561.812862214473
arrival 612.2424557911986
arrival 665.3367240000815
departure 666.2943800330871
arrival 741.9235063222254
arrival 759.1964167402336
arrival 777.9071463598152
arrival 805.1329999582114
arrival 805.2943357223605
arrival 896.919789339986
arrival 909.2140789827772
arrival 1039.7939168369373


6